# 画像分類の体験

[teachable machine](https://teachablemachine.withgoogle.com/) で画像分類をコーディングなしで実行できる。

2種類の画像を用意する。

例：犬と猫

[旧医科歯科大のサイト](http://drive.google.com/file/d/1b7sTWITljlC3zKk1SvggMZx0rbyojoWY/view?usp=sharing)にcovid, normalの肺X線画像があるので使ってみる。
ダウンロード(54MB)して展開する (images_TMDU.zip) images-> COVID-NORMAL にhealthy, covid19のフォルダがあり、116個の画像がある。


* 収集(複数の種類の画像をアップロードしてラベルをつけておく)
画像プロジェクト、標準の画像モデルを選び、Class1、Class2にファイルをアップロードする。

* トレーニング（アップロードしたらそのまま実行できる。詳細でハイパーパラメータ（エポック、バッチサイズ、学習率）を変更できる）

* プレビュー　webcam-> ファイルに変更すると、アップロードできるので、normal.jpg, covid.jpgをアップロードして
  正しい出力が得られることを確認する。
* 「エクスポートする」で学習モデルをダウンロードできる。TensorFlow, Kerasを選択。スニペットをコピーしてColabノートブックにペースト。
teachableフォルダにkeras_model.h5, labels.txtと画像(normal.jpg, covid.jpg)を置けば読み込める。

ただし、colabで実行しようすると、tensorflowのバージョンが違うのでそのままでは動かない。
!pip install tensorflow==2.10.0
を実行（時間がかかる）すると、セッションを再起動する必要があるが、アップロードしたファイルは残るので、そのまま実行できる。

In [1]:
# 必要なファイル　Keras_Model.h5, labels.txt, covid.jpg, normal.jpg をColabにアップロードする
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("teachable/keras_model.h5", compile=False) # windowsではkeras_model.h5
model.summary()

# Load the labels
class_names = open("teachable/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("teachable/covid.jpg").convert("RGB") # <IMAGE_PATH> を covid.jpg or normal.jpg に書き換える。

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_3 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_5 (Sequential)   (None, 2)                 128300    
                                                                 
Total params: 538,508
Trainable params: 524,428
Non-trainable params: 14,080
_________________________________________________________________
1/1 [==============================] - 2s 2s/step
Class: covid
Confidence Score: 0.9994518
